In [ ]:
import os 
from google.colab import drive

In [ ]:
!pip install tsmoothie

In [ ]:
import numpy as np
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import tsmoothie
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Conv1D,Input
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from sklearn.decomposition import PCA
from tensorflow.keras import optimizers
from sklearn.preprocessing import MinMaxScaler
import sklearn
from sklearn.metrics import mean_squared_error,r2_score
%matplotlib inline

In [ ]:
from TLSTM import TLSTM

In [ ]:
#LSTM=tf.compat.v1.keras.layers.CuDNNLSTM 
LSTM = tf.compat.v1.keras.layers.LSTM
GRU = tf.compat.v1.keras.layers.GRU
#GRU = tf.compat.v1.keras.layers.CuDNNGRU
a=tf.compat.v1.keras.layers.CuDNNLSTM
smoother=tsmoothie.KalmanSmoother(component='level_trend',  component_noise={'level':0.1, 'trend':0.1})

In [ ]:
data_path = "CNN_TLSTM dataset.csv"

In [ ]:
# loading data

data_1=pd.read_csv(data_path)
print(data_1.head())
print(data_1.columns)
data=data_1[['Open','High',
             'Low','Close','Nasdaq Index (NI)',
             'Dow-Jones  Industrial \nAverage (DJIA)',
             'Shanghai Composite Index (SCI)',
             'Hang Seng Index (HSI)']]
dates=data_1[['Date']]

# normalising data

Open=MinMaxScaler()
High=MinMaxScaler()
Low=MinMaxScaler()
Close=MinMaxScaler()
Volume=MinMaxScaler()
Lowest=MinMaxScaler()

dataaa=np.zeros((data.shape),dtype=np.float64)
a=Open.fit_transform(data[['Open']])
dataaa[:,0] = a[:,0]
a=High.fit_transform(data[['High']])
dataaa[:,1] = a[:,0]
a=Low.fit_transform(data[['Low']])
dataaa[:,2] = a[:,0]
a=Close.fit_transform(data[['Close']])
dataaa[:,3] = a[:,0]
a=Volume.fit_transform(data[['Nasdaq Index (NI)']])
dataaa[:,4] = a[:,0]
a=Lowest.fit_transform(data[['Dow-Jones  Industrial \nAverage (DJIA)']])
dataaa[:,5] = a[:,0]
a=Open.fit_transform(data[['Shanghai Composite Index (SCI)']])
dataaa[:,6] = a[:,0]
a=Open.fit_transform(data[['Hang Seng Index (HSI)']])

a=dataaa[:,0]
smoother.smooth(a)
dataaa[:,0]=smoother.smooth_data

a=dataaa[:,1]
smoother.smooth(a)
dataaa[:,1]=smoother.smooth_data

a=dataaa[:,2]
smoother.smooth(a)
dataaa[:,2]=smoother.smooth_data

a=dataaa[:,3]
smoother.smooth(a)
dataaa[:,3]=smoother.smooth_data

a=dataaa[:,4]
smoother.smooth(a)
dataaa[:,4]=smoother.smooth_data

a=dataaa[:,5]
smoother.smooth(a)
dataaa[:,5]=smoother.smooth_data

a=dataaa[:,6]
smoother.smooth(a)
dataaa[:,6]=smoother.smooth_data


full_data=dataaa

# finding total number of days

data=dataaa


total_days=data.shape[0]
print(total_days)

         Date      Open  ...  Shanghai Composite Index (SCI)  Hang Seng Index (HSI)
0  2003-12-01  1.203398  ...                          1432.7                12457.0
1  2003-12-02  1.196101  ...                          1435.3                12412.2
2  2003-12-03  1.209000  ...                          1445.0                12361.2
3  2003-12-04  1.212004  ...                          1452.8                12342.7
4  2003-12-05  1.207802  ...                          1451.1                12314.7

[5 rows x 9 columns]
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Nasdaq Index (NI)',
       'Dow-Jones  Industrial \nAverage (DJIA)',
       'Shanghai Composite Index (SCI)', 'Hang Seng Index (HSI)'],
      dtype='object')
4652


In [ ]:
training_period_dates=dates.loc[:int(full_data.shape[0]*1)]
#testing_period_dates=dates.loc[int(full_data.shape[0]*0.9):]

In [ ]:
total_days=data.shape[0]

timesteps=5

Train_data=data[:round(total_days*1),:]
#Test_data=data[round(total_days*0.9):,:]
print(Train_data.shape[0]," Days are used for Training")
#print(Test_data.shape[0]," Days are used for Testing")
# Function for ordering or making sequence of our test and train data according to date
def make_sequence(data,timesteps):
    X_train=[]
    Y_train=[]
    for i in range(len(data)-timesteps):
        X_train.append(data[i:(i+timesteps),[0,1,2,4,5,6,7]])
        Y_train.append(data[i+timesteps,3])
    return X_train,Y_train
Train_data,Train_Label=make_sequence(Train_data,timesteps)
Train_data,Train_Label=np.array(Train_data),np.array(Train_Label)
#Test_data,Test_Label=make_sequence(Test_data,timesteps)
#Test_data,Test_Label=np.array(Test_data),np.array(Test_Label)
input_shape=((Train_data).shape[1],(Train_data).shape[2])
print(input_shape)
Train_Label = Train_Label.reshape(-1,1)


4652  Days are used for Training
(5, 7)


In [ ]:
#compiling model
lr=0.001
mae = tf.keras.losses.MeanAbsoluteError()
model.compile(optimizer = optimizers.Adam(learning_rate= lr), loss = mae)
#fitting model
history=model.fit(Train_data,Train_Label,batch_size=60,epochs=50)

Epoch 1/60
128/128 [==============================] - 14s 67ms/step - loss: 0.0039
Epoch 2/60
128/128 [==============================] - 9s 67ms/step - loss: 4.8198e-04
Epoch 3/60
128/128 [==============================] - 9s 68ms/step - loss: 4.1844e-04
Epoch 4/60
128/128 [==============================] - 9s 68ms/step - loss: 4.1596e-04
Epoch 5/60
128/128 [==============================] - 9s 67ms/step - loss: 2.4331e-04
Epoch 6/60
128/128 [==============================] - 9s 68ms/step - loss: 1.9086e-04
Epoch 7/60
128/128 [==============================] - 9s 68ms/step - loss: 1.8562e-04
Epoch 8/60
128/128 [==============================] - 9s 67ms/step - loss: 1.3219e-04
Epoch 9/60
128/128 [==============================] - 9s 67ms/step - loss: 1.2021e-04
Epoch 10/60
128/128 [==============================] - 9s 67ms/step - loss: 1.1121e-04
Epoch 11/60
128/128 [==============================] - 9s 67ms/step - loss: 9.0183e-05
Epoch 12/60
128/128 [==============================] - 

In [ ]:
#predicting values from our model of test data
Test_Predicted_Label=model.predict(Train_data)
print(Test_Predicted_Label.shape)
print(Test_Predicted_Label)
Test_Label = Train_Label.reshape(Train_Label.shape[0],1)
y_hat_test = Close.inverse_transform(Test_Predicted_Label)
# Save the predicted values
np.savetxt("predicted.csv", y_hat_test, delimiter=",")

(4593, 1)
[[0.34392822]
 [0.34930035]
 [0.35225567]
 ...
 [0.23487885]
 [0.23384579]
 [0.2318861 ]]
